# PLEXsim 결과 `HDF5` 파일 분석 튜토리얼

**Simulation 저장 그룹 구조**   
(e.g. `proj.h5`)

```
/settings
  /environment
  /grids/{grid_index}
  /simulation
    @delta_time
      ...
/cycles/{cycle}
  /stats
    @n_particles
    @kinetic_E
    @field_E
    @total_E
  /field
    B
    E
  /grids/{grid_index}
    @tracking_particles
    X  # might be virtual datatset to Grid
    U  # might be virtual datatset to Grid
    /tracked
      @tracking_ids
      X
      U
```

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FrTxWJ%2FbtqVb1q5svj%2FUfvALrkvpXcfsLefvIgeKK%2Fimg.png' width='50%'></img>

## 01 초기 세팅 확인하기

In [ ]:
fp = 'sample/test.h5'

In [ ]:
import h5py

h5f = h5py.File(fp, 'r')

- `keys()` 메서드로 하위 항목을 확인할 수 있다.

In [ ]:
h5f.keys()

- `[]` 안에 경로를 넣어 하위 항목으로 이동할 수 있다.

In [ ]:
h5f['settings'].keys()

In [ ]:
h5f['settings/environment']

In [ ]:
h5f['settings']['environment']

- `.attrs` 로 메타데이터(attribute)를 읽을 수 있고, `dict()`를 사용하면 파이썬 딕셔너리로 변환 가능하다.

In [ ]:
dict(h5f['settings/environment'].attrs)

In [ ]:
h5f['settings/environment'].attrs['grid_shape']

In [ ]:
dict(h5f['settings/grids/0'].attrs)

In [ ]:
h5f.close()

- `with as` 구문을 사용하면 자동으로 file close를 해주어 실수를 줄일 수 있다.

In [ ]:
with h5py.File(fp, 'r') as h5f:
    print(h5f['settings'].keys())

## 02 입자 위치 및 속도 읽고 변환하기

#### 특정 cycle, grid의 입자 위치, 속도 배열을 읽기

In [ ]:
import h5py

fp = 'sample/test.h5'

h5f = h5py.File(fp, 'r')

In [ ]:
cycles = map(int, h5f['cycles'].keys())
cycle = sorted(cycles)[-1]
cycle

In [ ]:
grid = 0
_path = f'cycles/{cycle}/grids/{grid}/X'

h5f[_path]

- dataset에 인덱스로 접근해 값을 읽어올 수 있다.

In [ ]:
h5f[_path][287, 0]

In [ ]:
X = h5f[_path][:]
print(type(X))
print(X.shape)

#### `Numpy` 배열을 `CSV`로 저장하기

In [ ]:
import numpy as np

np.savetxt('test.csv', X, delimiter=',')

#### `Numpy` 배열을 `Pandas` DataFrame으로 바꾸기

In [ ]:
import pandas as pd

pd.DataFrame(X)

In [ ]:
h5f.close()

#### `multiprocessing`의 `Pool`을 이용해 모든 cycle, grid의 입자 위치, 속도를 `CSV`로 저장하는 코드

In [ ]:
%%time

import os
from multiprocessing import Pool

import h5py
import numpy as np


fp = 'sample/test.h5'
out_dir = 'out/'
n_process = 32

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

with h5py.File(fp, 'r') as h5f:
    def save_particles(cycle):
        _path = f'cycles/{cycle}/grids/0/X'
        if _path not in h5f:
            return

        for grid in h5f[f'cycles/{cycle}/grids'].keys():
            prefix = f'cycles/{cycle}/grids/{grid}'
            filepath = out_dir + prefix.replace('/', '_')

            X = h5f[f'{prefix}/X'][:]
            np.savetxt(f'{filepath}_X.csv', X, delimiter=',')

            U = h5f[f'{prefix}/U'][:]
            np.savetxt(f'{filepath}_U.csv', U, delimiter=',')
            
    cycles = map(int, h5f['cycles'].keys())

    with Pool(n_process) as p:
        p.map(save_particles, cycles)

## 03 입자 속도 분포를 그래프로 그리기

In [ ]:
import h5py

fp = 'sample/test.h5'
_path = 'cycles/0/grids/0/U'

h5f = h5py.File(fp, 'r')
U = h5f[_path][:]

In [ ]:
import pandas as pd

U_df = pd.DataFrame(U, columns=['vx', 'vy', 'vz'])
U_df['v'] = np.linalg.norm(U, axis=1)
U_df

In [ ]:
import plotly.express as px

px.histogram(U_df, x='v', marginal='box')

In [ ]:
h5f.close()

## 04 추적 중인 입자의 위치와 속도 변화를 그래프로 그리기

In [ ]:
import h5py

fp = 'sample/test.h5'

h5f = h5py.File(fp, 'r')

In [ ]:
grid = 0
tracking_id = 2

history = []
for cycle in sorted(h5f['cycles'], key=int)[:-1]:
    base_path = f'cycles/{cycle}/grids/{grid}/tracked'
    if base_path not in h5f:
        continue

    ids = h5f[base_path].attrs['tracking_ids']
    if tracking_id not in ids:
        break
    index = np.where(ids == tracking_id)[0].item()
    
    X = h5f[f'{base_path}/X'][index]
    U = h5f[f'{base_path}/U'][index]
    history.append((X, U))

In [ ]:
# !pip install pandas

In [ ]:
import pandas as pd

X_df = pd.DataFrame([X for X, U in history], columns=['x', 'y', 'z'])
U_df = pd.DataFrame([U for X, U in history], columns=['vx', 'vy', 'vz'])
X_df

In [ ]:
import plotly.graph_objects as go

fig1 = go.Figure()
for x in ['x', 'y', 'z']:
    fig1.add_trace(go.Scatter(x=X_df.index, y=X_df[x], name=x))
fig1.update_layout(title='position')
fig1.show()

fig2 = go.Figure()
for v in ['vx', 'vy', 'vz']:
    fig2.add_trace(go.Scatter(x=U_df.index, y=U_df[v], name=v))
fig2.update_layout(title='velocity')
fig2.show()

In [ ]:
h5f.close()